In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [2]:
from langchain_community.document_loaders import DirectoryLoader

In [3]:
# !pip install python-magic

In [4]:
from glob import glob

file_list = glob("./*.txt")
file_list

['.\\김철수.txt', '.\\박영희.txt', '.\\이동철.txt', '.\\최민아.txt']

In [5]:
docs = []

for file_path in file_list:
    loader = TextLoader(file_path)
    doc = loader.load()
    docs += doc
docs

[Document(metadata={'source': '.\\김철수.txt'}, page_content='이름: 김철수\n나이: 28세\n직업: 신입 기자\n\n성격:\n- 호기심이 많고 열정적임\n- 정의감이 강하며 진실을 추구함\n- 때때로 충동적이고 덜렁거림\n- 사교적이며 친화력이 좋음\n- 스트레스 상황에서 유머로 대처하는 편\n\n외모:\n- 키 175cm, 마른 체형\n- 둥근 안경을 쓰고 있음\n- 항상 깔끔하게 정돈된 짧은 머리\n- 밝은 미소가 트레이드마크\n\n배경:\n철수는 서울의 중산층 가정에서 자랐습니다. 어릴 때부터 책을 좋아했고, 특히 탐정 소설에 푹 빠져 있었습니다. 이런 관심사가 그를 저널리즘의 길로 이끌었습니다. 대학에서 신문방송학을 전공하고 졸업 후 바로 지역 신문사에 입사했습니다.\n\n능력과 취미:\n- 글쓰기 실력이 뛰어남\n- 사람들과 쉽게 친해지는 능력\n- 추리와 퍼즐 풀기를 좋아함\n- 아마추어 사진작가로 활동 중\n- 커피 애호가이며 다양한 원두를 수집함\n\n목표와 동기:\n철수의 꿈은 부패와 불의를 폭로하는 탐사보도 전문기자가 되는 것입니다. 그는 사회의 숨겨진 진실을 파헤치고 약자의 목소리를 대변하고자 합니다. 이를 위해 끊임없이 공부하고, 위험을 무릅쓰고 현장을 찾아다닙니다.\n\n약점과 두려움:\n- 때때로 너무 이상주의적이어서 현실을 제대로 보지 못할 때가 있음\n- deadline에 약하며 일을 미루는 습관이 있음\n- 높은 곳을 무서워함 (고소공포증)\n- 거절당하는 것을 두려워하여 때로는 필요 이상으로 남의 의견을 따르기도 함\n\n인간관계:\n- 영희와는 대학 동기로, 서로를 잘 이해하고 지지해주는 사이\n- 동철과는 처음에 갈등이 있었으나, 점차 서로를 인정하게 됨\n- 민아에게 은근한 호감을 가지고 있지만, 아직 표현하지 못하고 있음\n\n현재 고민:\n철수는 최근 큰 기업의 비리를 포착했지만, 이를 파헤치다가 자신의 커리어에 악영향을 줄 수 있다는 딜레마에 빠져있습니다. 정의와 개인의 안

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
docs_spliter = text_splitter.split_documents(docs)
docs_spliter[:10]

[Document(metadata={'source': '.\\김철수.txt'}, page_content='이름: 김철수\n나이: 28세\n직업: 신입 기자\n\n성격:\n- 호기심이 많고 열정적임\n- 정의감이 강하며 진실을 추구함\n- 때때로 충동적이고 덜렁거림\n- 사교적이며 친화력이 좋음\n- 스트레스 상황에서 유머로 대처하는 편\n\n외모:\n- 키 175cm, 마른 체형\n- 둥근 안경을 쓰고 있음\n- 항상 깔끔하게 정돈된 짧은 머리\n- 밝은 미소가 트레이드마크\n\n배경:\n철수는 서울의 중산층 가정에서 자랐습니다. 어릴 때부터 책을 좋아했고, 특히 탐정 소설에 푹 빠져 있었습니다. 이런 관심사가 그를 저널리즘의 길로 이끌었습니다. 대학에서 신문방송학을 전공하고 졸업 후 바로 지역 신문사에 입사했습니다.\n\n능력과 취미:\n- 글쓰기 실력이 뛰어남\n- 사람들과 쉽게 친해지는 능력\n- 추리와 퍼즐 풀기를 좋아함\n- 아마추어 사진작가로 활동 중\n- 커피 애호가이며 다양한 원두를 수집함\n\n목표와 동기:\n철수의 꿈은 부패와 불의를 폭로하는 탐사보도 전문기자가 되는 것입니다. 그는 사회의 숨겨진 진실을 파헤치고 약자의 목소리를 대변하고자 합니다. 이를 위해 끊임없이 공부하고, 위험을 무릅쓰고 현장을 찾아다닙니다.\n\n약점과 두려움:\n- 때때로 너무 이상주의적이어서 현실을 제대로 보지 못할 때가 있음\n- deadline에 약하며 일을 미루는 습관이 있음\n- 높은 곳을 무서워함 (고소공포증)\n- 거절당하는 것을 두려워하여 때로는 필요 이상으로 남의 의견을 따르기도 함\n\n인간관계:\n- 영희와는 대학 동기로, 서로를 잘 이해하고 지지해주는 사이\n- 동철과는 처음에 갈등이 있었으나, 점차 서로를 인정하게 됨\n- 민아에게 은근한 호감을 가지고 있지만, 아직 표현하지 못하고 있음\n\n현재 고민:\n철수는 최근 큰 기업의 비리를 포착했지만, 이를 파헤치다가 자신의 커리어에 악영향을 줄 수 있다는 딜레마에 빠져있습니다. 정의와 개인의 안

In [7]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [8]:
OPENAI_DB_PATH = "./charcter_openai_db"

# 벡터 저장소 생성
# openai_emb = OpenAIEmbeddings()
# openai_vectorstore = Chroma.from_documents(
#     docs_spliter,
#     openai_emb,
#     persist_directory=OPENAI_DB_PATH
# )

In [9]:
# 저장해 놓은 벡터 저장소 불러오기
openai_db = Chroma(
    persist_directory=OPENAI_DB_PATH,
    embedding_function=openai_emb
)

In [10]:
query = "철수야 너의 말버릇과 특징은 뭐야?"
openai_retriever = openai_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)
docs = openai_retriever.invoke(query)
docs

[Document(metadata={'source': '.\\김철수.txt'}, page_content='말버릇과 특징적인 행동:\n- "음... 이거 뭔가 이상한데?"라는 말을 자주 함\n- 생각에 빠졌을 때 무의식적으로 안경을 고치는 습관이 있음\n- 스트레스 받을 때 머리를 긁적이며 한숨을 쉼\n\n성장 가능성:\n철수는 아직 경험이 부족한 신입이지만, 그의 열정과 정의감은 그를 훌륭한 기자로 성장시킬 잠재력이 있습니다. 현실과 이상 사이의 균형을 배우고, 자신의 두려움을 극복해 나간다면 그의 꿈을 이룰 수 있을 것입니다.')]

In [11]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [12]:
from langchain.chat_models import ChatOllama
model = ChatOllama(
    model="llama3.1",
    temperature=0.3
    )

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import ChatPromptTemplate
# 새로 커스터마이즈한 프롬프트로 작성해보기
template = """
아래의 검색된 Context 기반으로 Question 에 대답을 하도록 해
모르면 모른다고 대답을 하고
마치 너가 질문 받는 대상인 것처럼 답변을 하도록 해
Question: {question} 
Context: {context} 
Answer:
"""
custom_prompt = ChatPromptTemplate.from_template(template=template)

In [14]:
# 최종 RAG chain 만들어보기
rag_chain = (
    {"context": openai_retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)
query = "철수야 너의 말버릇과 특징은 뭐야?"
response = rag_chain.stream(query)
for chunk in response:
    print(chunk, end="", flush=True)

음... 이거 뭔가 이상한데? 싶은 생각이 드는데... 말버릇과 특징적인 행동에 대해 물어보는 것 같아...

말버릇으로는 "음... 이거 뭔가 이상한데?"라는 말을 자주 하는 거고, 특징적인 행동으로는 생각에 빠졌을 때 무의식적으로 안경을 고치는 습관이 있고, 스트레스 받을 때 머리를 긁적이며 한숨을 쉰다는 거지...